In [107]:
import pandas as pd
import dotenv
import os
dotenv.load_dotenv()

MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])



In [108]:
from nautilus_trader.data.engine import ParquetDataCatalog

print(CATALOG_PATH)
catalog = ParquetDataCatalog(CATALOG_PATH)


/Users/tobiaspucher/GitHub/nautilus/catalog


In [109]:
catalog.bars()

[Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.00730,1.02780,1.00540,1.02460,6642,946857600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02430,1.03400,1.02130,1.02920,7339,946944000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02930,1.04020,1.02840,1.03260,6570,947030400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03250,1.04150,1.02720,1.03300,7223,947116800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03290,1.03320,1.02600,1.02980,5689,947203200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02860,1.03050,1.02110,1.02510,6197,947462400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02530,1.03410,1.02460,1.03360,6637,947548800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03370,1.03700,1.02610,1.03080,6877,947635200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03090,1.03160,1.02370,1.02600,6176,947721600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02610,1.02750,1.01140,1.01220,6176,947808000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.01290,1.01480,1.00690,

In [110]:
import data_utils

In [111]:
from nautilus_trader.model.identifiers import Venue, InstrumentId
from nautilus_trader.config import BacktestVenueConfig
from nautilus_trader.persistence.loaders import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.model.data import BarType, Bar
from nautilus_trader.core.datetime import dt_to_unix_nanos

start = dt_to_unix_nanos(pd.Timestamp("2001-01-01", tz="UTC"))
end =  dt_to_unix_nanos(pd.Timestamp("2002-11-23", tz="UTC"))

bar_type:BarType = BarType.from_str("EURUSD.SIM-1-DAY-LAST-EXTERNAL")
instrument_id:InstrumentId = bar_type.instrument_id


In [112]:
str(instrument_id.value)

'EURUSD.SIM'

In [113]:
from nautilus_trader.model.instruments import Instrument

instrument:Instrument = catalog.instruments()[0]

In [114]:
instrument

CurrencyPair(id=EURUSD.SIM, raw_symbol=EURUSD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)

In [115]:
bar_type

BarType(EURUSD.SIM-1-DAY-LAST-EXTERNAL)

In [116]:
catalog.bars()

[Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.00730,1.02780,1.00540,1.02460,6642,946857600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02430,1.03400,1.02130,1.02920,7339,946944000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02930,1.04020,1.02840,1.03260,6570,947030400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03250,1.04150,1.02720,1.03300,7223,947116800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03290,1.03320,1.02600,1.02980,5689,947203200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02860,1.03050,1.02110,1.02510,6197,947462400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02530,1.03410,1.02460,1.03360,6637,947548800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03370,1.03700,1.02610,1.03080,6877,947635200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03090,1.03160,1.02370,1.02600,6176,947721600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02610,1.02750,1.01140,1.01220,6176,947808000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.01290,1.01480,1.00690,

In [120]:


venue_configs = [
    BacktestVenueConfig(
        name="SIM",
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10000 USD"],
    ),
]

from nautilus_trader.config import BacktestDataConfig

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=Bar,
        instrument_id=instrument_id.value,
        start_time=start,
        end_time=end,
    ),
]
data_configs


from nautilus_trader.trading.strategy import ImportableStrategyConfig

from decimal import Decimal

strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config=dict(
            instrument_id=instrument.id.value,
            bar_type=data_utils.bar_type_to_str(bar_type),
            fast_ema_period=10,
            slow_ema_period=20,
            trade_size=Decimal(100_000),
        ),
    ),
]

from nautilus_trader.config import BacktestRunConfig, BacktestEngineConfig

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)]

from nautilus_trader.backtest.node import BacktestNode

node = BacktestNode(configs)

In [121]:
node.run()


Failed to set global default dispatcher because of error: a global default trace dispatcher has already been set
2023-11-30T11:19:04.306951000Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2023-11-30T11:19:04.306973000Z [INF] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2023-11-30T11:19:04.306978000Z [INF] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2023-11-30T11:19:04.306981000Z [INF] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2023. All rights reserved.
2023-11-30T11:19:04.306982000Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2023-11-30T11:19:04.306982001Z [INF] BACKTESTER-001.BacktestEngine: 
2023-11-30T11:19:04.306983000Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2023-11-30T11:19:04.306984000Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2023-11-30T

[BacktestResult(trader_id='BACKTESTER-001', machine_id='Tobiass-MacBook-Air.local', run_config_id='20136de6fc3acfbb3cf50626c7a307351faa6be6841dc0b1da153fcf59be4e79', instance_id='14dfeaa1-2e87-47ce-95a6-065fbd82f512', run_id='28dd1f86-9209-4ef9-92a7-a3a2b94495f7', run_started=1701343144331421000, run_finished=1701343144371027000, backtest_start=978307200000000000, backtest_end=1037923200000000000, elapsed_time=59616000.0, iterations=0, total_events=120, total_orders=40, total_positions=20, stats_pnls={'USD': {'PnL (total)': 4977.19, 'PnL% (total)': 49.77190000000001, 'Max Winner': 10544.32, 'Avg Winner': 2610.6237499999997, 'Min Winner': 114.11, 'Min Loser': -215.65, 'Avg Loser': -1325.6499999999999, 'Max Loser': -2425.53, 'Expectancy': 248.85950000000003, 'Win Rate': 0.4}}, stats_returns={'Returns Volatility (252 days)': 0.0923799969747356, 'Average (Return)': 0.0031938743414896644, 'Average Loss (Return)': -0.014368238059648044, 'Average Win (Return)': 0.02953704294319623, 'Sharpe Ra

0.000000000Z [INF] BACKTESTER-001.EMACross-000: <--[EVT] OrderFilled(instrument_id=EURUSD.SIM, client_order_id=O-20011206-0000-001-000-14, venue_order_id=SIM-1-014, account_id=SIM-001, trade_id=SIM-1-271, position_id=SIM-1-007, order_side=BUY, order_type=MARKET, last_qty=2166, last_px=0.89450 USD, commission=0.04 USD, liquidity_side=TAKER, ts_event=1007596800000000000).
2001-12-06T00:00:00.000000000Z [INF] BACKTESTER-001.Portfolio: EURUSD.SIM net_position=-97834
2001-12-06T00:00:00.000000000Z [INF] BACKTESTER-001.Portfolio: EURUSD.SIM margin_maint=2_650.59 USD
2001-12-06T00:00:00.000000000Z [INF] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[AccountBalance(total=14_792.12 USD, locked=2_650.59 USD, free=12_141.53 USD)], margins=[MarginBalance(initial=0.00 USD, maintenance=2_650.59 USD, instrument_id=EURUSD.SIM)], event_id=58be43a6-deec-4bc5-966e-28e3584d8714).
2001-12-06T00:00:00.000000000Z [INF] B

In [122]:
catalog.instruments()

[CurrencyPair(id=EURUSD.SIM, raw_symbol=EURUSD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None),
 CurrencyPair(id=GBPUSD.SIM, raw_symbol=GBPUSD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None),
 CurrencyPair(id=USDJPY.SIM, raw_symbol=USDJPY, asset_class=FX, asset_type=SPOT, quote_currency=JPY, is_inverse=False, price_precision=3, price_increment=0.001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [123]:
catalog.bars()

[Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.00730,1.02780,1.00540,1.02460,6642,946857600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02430,1.03400,1.02130,1.02920,7339,946944000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02930,1.04020,1.02840,1.03260,6570,947030400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03250,1.04150,1.02720,1.03300,7223,947116800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03290,1.03320,1.02600,1.02980,5689,947203200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02860,1.03050,1.02110,1.02510,6197,947462400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02530,1.03410,1.02460,1.03360,6637,947548800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03370,1.03700,1.02610,1.03080,6877,947635200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03090,1.03160,1.02370,1.02600,6176,947721600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02610,1.02750,1.01140,1.01220,6176,947808000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.01290,1.01480,1.00690,